## Importing Libraries

In [ ]:
!pip install split-folders

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab import drive
from IPython.display import display, Image
import matplotlib.image as mpimg
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.optimizers import SGD, Adam
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import splitfolders

## Reading Data

In [ ]:
# Accessing My Google Drive
drive.mount('/content/drive')

In [ ]:
# !unzip "/content/drive/MyDrive/CV Project/train.zip" -d "/content/drive/MyDrive/CV Project/Train/"

In [ ]:
path = '/content/drive/MyDrive/CV Project/Train/train/'
out_path = '/content/drive/MyDrive/data'

In [ ]:
# import splitfolders

# splitfolders.ratio(path, output=out_path, seed=1337, ratio=(.8, 0.1,0.1), 
#                    group_prefix=None)

In [ ]:
TRAINING_DIR = '/content/drive/MyDrive/data/train'
Validation_DIR = '/content/drive/MyDrive/data/val'
Test_DIR = '/content/drive/MyDrive/data/test'

## EDA

In [ ]:
os.listdir(path)

In [ ]:
# directory[0]

In [ ]:
# for i, file in enumerate(os.listdir(path + directory)):
#   print(i)

In [ ]:
activity_map = {'c0': 'Safe driving', 
                'c1': 'Texting - right', 
                'c2': 'Talking on the phone - right', 
                'c3': 'Texting - left', 
                'c4': 'Talking on the phone - left', 
                'c5': 'Operating the radio', 
                'c6': 'Drinking', 
                'c7': 'Reaching behind', 
                'c8': 'Hair and makeup', 
                'c9': 'Talking to passenger'}


plt.figure(figsize = (12, 20))
image_count = 1

for directory in os.listdir(path):
        for i, file in enumerate(os.listdir(path + directory)):
            if i == 1:
                break
            else:
                fig = plt.subplot(5, 2, image_count)
                image_count += 1
                image = mpimg.imread(path + directory + '/' + file)
                plt.imshow(image)
                plt.title(f"{activity_map[directory]}----{image.shape}")

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/CV Project/driver_imgs_list.csv')
data

In [ ]:
import plotly.express as px

px.histogram(data, x="classname", color="classname", title="Number of images by categories ")

## Baseline Dense layers model

## Baseline CNN model

In [ ]:
batch_size = 64
nb_epoch = 6

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  # rotation_range=30,
                                  # width_shift_range=0.2,
                                  # height_shift_range=0.2,
                                  # shear_range=0.2,
                                  # zoom_range=0.2,
                                  # horizontal_flip=True,
                                  # fill_mode='nearest'
                                   )


train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=batch_size,
                                                      class_mode='categorical',
                                                      target_size=(480, 640))
  
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(directory=Validation_DIR,
                                                              batch_size = batch_size,
                                                              class_mode='categorical',
                                                              target_size=(480, 640))

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(480, 640, 3)),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),

                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    
                                    tf.keras.layers.Conv2D(128, (5,5), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    
                                    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),

                                    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation='relu'),
                                    tf.keras.layers.Dense(256, activation='relu'),
                                    tf.keras.layers.Dense(10, activation='softmax')
                                  ])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    validation_data=validation_generator)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = validation_datagen.flow_from_directory(directory=Test_DIR,
                                                              batch_size = batch_size,
                                                              class_mode='categorical',
                                                              target_size=(480, 640))

In [ ]:
score1 = model.evaluate(test_generator, verbose=1)
print('Loss: ', score1[0])
print('Accuracy: ', score1[1]*100, ' %')

In [ ]:
model.save('/content/drive/MyDrive/CV Project')

In [ ]:
model.save_weights('/content/drive/MyDrive/CV Project')

## Data augmentation

In [ ]:
model = load_model('/content/drive/MyDrive/CV Project')

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1.0/255, 
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   horizontal_flip = True, 
                                   validation_split = 0.2)

train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=batch_size,
                                                      class_mode='categorical',
                                                      target_size=(480, 640))

test_datagen = ImageDataGenerator(rescale=1.0/ 255)

validation_generator = test_datagen.flow_from_directory(directory=Validation_DIR,
                                                              batch_size = batch_size,
                                                              class_mode='categorical',
                                                              target_size=(480, 640))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
nb_train_samples = 17934
nb_validation_samples = 2237

In [ ]:
history_v2 = model.fit(train_generator,
                         steps_per_epoch = nb_train_samples // batch_size,
                         epochs = nb_epoch, 
                         verbose = 1,
                         validation_data = validation_generator,
                         validation_steps = nb_validation_samples // batch_size)

In [ ]:
model.save('/content/drive/MyDrive/CV Project/Model2')  

## Transfer Learning

## CNN visualization